In [ ]:
import os
import sys
from pprint import pprint
import json
from notion_client import Client
from dotenv import load_dotenv

load_dotenv()

NOTION_TOKEN = os.getenv("NOTION_TOKEN", "")

while NOTION_TOKEN == "":
    print("NOTION_TOKEN not found.")
    NOTION_TOKEN = input("Enter your integration token: ").strip()

notion = Client(auth=NOTION_TOKEN)

In [ ]:
from pprint import pprint

list_users_response = notion.users.list()
pprint(list_users_response)

In [ ]:
def get_page(name):
  return notion.databases.query(
    **{
          "database_id": os.getenv("database_id",""),
          "filter": {
              "property": "Name",
              "title":{
                "equals": name,
              },
          },
      }
)


In [ ]:
def get_page_id(retrieved_page):
  return (retrieved_page["results"][0]["url"]).split("-")[-1]

In [ ]:
get_page("TSx.jl")

In [ ]:
TSx_url = get_page("TSx.jl")
TSx_id = get_page_id(TSx_url)

In [ ]:
TSx_page_id = notion.pages.retrieve(page_id=TSx_id)["url"]

In [ ]:
TSx_page = notion.blocks.children.list(TSx_id)

In [ ]:
pprint(TSx_page)

In [ ]:
TSx_page["results"]

In [ ]:
def print_paragraph(page):
  for paragraph in page["results"]:
    paragraph_type = paragraph["type"]
    pprint(paragraph[paragraph_type])["rich_text"][1]["plain_text"]
    if paragraph["has_children"]:
      paragraph_id=paragraph["id"]
      paragraph_children = notion.blocks.children.list(paragraph_id)

In [ ]:
pprint(TSx_page["results"][0])

In [ ]:
pprint(TSx_page["results"][5])

In [ ]:
pprint(notion.blocks.children.list(TSx_page["results"][1]["id"])["results"])

In [ ]:
#have to figure out how to update or append new paragraph
notion.blocks.update?
notion.blocks.children.append?

In [ ]:
notion.blocks.children.append?

In [ ]:
def parse_text_with_code(sentence):
  """
  when sentence like:
  1. implement `Base.getproperty` function for `TS`
  2. the test `log.(ts) == TS(log.(ts.x1), log.(ts.x2), date)` should pass and `log.(ts[:,[:x,:y]])` should also work
  should convert the `` to code
  """
  #r'''((?:[^;"']|"[^"]*"|'[^']*')+)'''
  words = re.split('(`[^`]*`)', sentence)
  parse_code_block = lambda hascode, content: {'annotations': {'code': hascode},
                           'text': {'content': content},
                           'type': 'text'}

  parsed_res = [parse_code_block(True, w[1:-1]) if (w[0] == "`" and w[-1] == "`") else parse_code_block(False, w) for w in words ]
  return parsed_res

parse_text_with_code("the test `log.(ts) == TS(log.(ts.x1), log.(ts.x2), date)` should pass and `log.(ts[:,[:x,:y]])` should also work")

In [ ]:
def parse_input(title,**kwargs):
  if title.find("`") == -1:
    toggle_template = [{'type': 'toggle',
    'toggle': {'rich_text': [{'type': 'text',
       'text': {'content': title, 'link': kwargs.get("link",None)}}],
     'color': 'default',
     'children': [{'type': 'code',
       'code': {'rich_text': [{'type': 'text',
          'text': {'content': kwargs.get("code", "should be filled in with example")}}],
        'language': kwargs.get("language", "julia")}}]}}]
  else:
    parsed_res = parse_text_with_code(title)
    toggle_template = [{'type': 'toggle',
    'toggle': {'rich_text': parsed_res,
     'color': 'default',
     'children': [{'type': 'code',
       'code': {'rich_text': [{'type': 'text',
          'text': {'content': kwargs.get("code", "should be filled in with example")}}],
        'language': kwargs.get("language", "julia")}}]}}]
  return toggle_template

In [ ]:
parse_input("the test should also work")

In [ ]:
parse_input("the test `log.(ts) == TS(log.(ts.x1), log.(ts.x2), date)` should pass and `log.(ts[:,[:x,:y]])` should also work")

In [ ]:
s = "the test `log.(ts) == TS(log.(ts.x1), log.(ts.x2), date)` should pass and `log.(ts[:,[:x,:y]])` should also work"
new_item = parse_input(s, code="ts.x == ts[:,:x]")
new_item

In [ ]:
notion.blocks.children.append(TSx_id, children=new_item)